In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from pathlib import Path
import random
from PIL import Image
import cv2
import torch
import numpy as np
# from torch.utils.data import DataLoader
# from torch.utils.data import Dataset
from torchvision import models
import torchvision.transforms as T

#import segmentation_models_pytorch as smp
#import albumentations as albu

In [3]:
# Data directory
loc = '/content/gdrive/MyDrive/Deeplobe GIT/deeplobe-ai-master/Datasets/semantic'
path = Path(loc)
path_lbl = path/'Masks'
path_img = path/'Images'
mask_definitions = path/'mask_definitions.json'

In [4]:
def return_files(path):
  files = []
  for ext in extensions:
    files.extend(path.glob('*.'+ext))
  return files

In [5]:
#get filenames from directory
extensions = ["jpg",'jpeg','png','PNG']
img_files = return_files(path_img)
mask_files = return_files(path_lbl)

In [7]:
#train val split

dataset_size = len(img_files)
valid_size = int(dataset_size * 0.5)
train_size = dataset_size - valid_size

#images
valid_imgs = random.sample(img_files,valid_size)
train_imgs = [img for img in img_files if img not in valid_imgs]

#corresponding masks
valid_img_names = [img.stem for img in valid_imgs]
valid_masks = [mask for mask in mask_files if mask.stem in valid_img_names]
train_masks = [mask for mask in mask_files if mask not in valid_masks]

In [8]:
#get label names and color-codes from mask_definitions json
with open(mask_definitions) as md:
    mask_def = eval(md.read())

mask_info = {}
label_dict = {}

for image in mask_def['masks']:
    mask=mask_def['masks'][image]['mask']
    mask = mask.lstrip('masks/')
    colors = mask_def['masks'][image]['color_categories']
    rgb = colors.keys()
    categories = [i['super_category'] for i in colors.values()]
    color_map = dict(zip(rgb,categories))
    mask_info.update({mask:color_map})

for v in mask_info.values():
  label_dict.update(v.items())

#create an rgb dict with colors as keys and labels as values
label_dict['(0, 0, 0)'] = 'background'

name2id = {v:k for k,v in enumerate(sorted(label_dict.values()))}
class_dict = {eval(v):k for k,v in enumerate(sorted(label_dict.keys()))}

In [9]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, img_files, img_transform=None,mask_transform=None):
        self.img_files = img_files
        self.img_transform = img_transform
        self.mask_transform = mask_transform

    #helper function to convert rbg mask to label encoded masks
    def mask_to_class(self,mask):
        target = torch.from_numpy(mask)
        #target = target.permute(1,2,0).contiguous()
        h,w = target.shape[0],target.shape[1]
        masks = torch.zeros(h, w, dtype=torch.long)
        colors = torch.unique(target.view(-1,target.size(2)),dim=0).numpy()
        target = target.permute(2, 0, 1).contiguous()
        mapping = class_dict #{tuple(c): t for c, t in zip(colors.tolist(), range(len(colors)))}

        for k in mapping:
            idx = (target==torch.tensor(k, dtype=torch.uint8).unsqueeze(1).unsqueeze(2))
            validx = (idx.sum(0) == 3) 
            masks[validx] = torch.tensor(mapping[k], dtype=torch.long)
        return masks

    def __getitem__(self, idx):
        image_file = self.img_files[idx]
        mask_file = path_lbl/image_file.name      
        # mask_file = str(path_lbl/image_file.name)
        # image_file = str(image_file)

        image = Image.open(image_file)
        mask = Image.open(mask_file)  

        # image = cv2.imread(image_file)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # mask = cv2.imread(mask_file)
        # mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        
        #apply tranforms
        if self.img_transform:
          image = self.img_transform(image)

        if self.mask_transform:
          mask = self.mask_transform(mask)

        #encode RGB mask to class_labels
        mask = np.array(mask)
        mask = self.mask_to_class(mask)

        return image, mask

    def __len__(self):  # return count of sample we have
        return len(self.img_files)

In [10]:
def get_img_transform(train=True):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.Resize(256))
    transforms.append(T.CenterCrop(224))
    transforms.append(T.ToTensor())
    transforms.append(T.Normalize(mean=[0.485,0.456,0.406],
                                std=[0.229,0.224,0.225]))

    return T.Compose(transforms)

def get_mask_transform(train=True):
    transforms = []
    # apply same resize and cropping on masks
    transforms.append(T.Resize(256))
    transforms.append(T.CenterCrop(224))    
    #transforms.append(T.ToTensor())

    return T.Compose(transforms)


In [11]:
# Datasets
train_dataset = CustomDataset(train_imgs,img_transform=get_img_transform(train=True),mask_transform=get_mask_transform(train=True))
valid_dataset = CustomDataset(valid_imgs,img_transform=get_img_transform(train=False),mask_transform=get_mask_transform(train=False))

In [29]:
train_dataset[0][0].shape

torch.Size([3, 224, 224])

In [12]:
# Dataloaders
bs = 1 #batch_size
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=bs,shuffle=True,num_workers=4)
valid_loader = torch.utils.data.DataLoader(valid_dataset,batch_size=bs,shuffle=False,num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [13]:
a=iter(train_loader)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [20]:
b=next(a)

In [23]:
b[0].shape

torch.Size([1, 3, 224, 224])

In [ ]:
#Model instantiation

classes = len(name2id.keys())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#FCN Head
from torchvision.models.segmentation.fcn import FCNHead

# create segmentation model with pretrained encoder
model = models.segmentation.fcn_resnet101(pretrained=True)
# change the output layer
model.classifier = FCNHead(2048,classes)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


Downloading: "https://download.pytorch.org/models/fcn_resnet101_coco-7ecb50ca.pth" to /root/.cache/torch/hub/checkpoints/fcn_resnet101_coco-7ecb50ca.pth


In [ ]:
# training the model:

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adadelta(params, lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
num_epochs = 2
total_batches = len(train_loader)

for epoch in range(num_epochs):
  model.train()
  model.to(device)
  i = 0
  for imgs,masks in train_loader:
    imgs = imgs.to(device)
    masks = masks.to(device)
    outputs = model(imgs)
    loss = criterion(outputs['out'],masks)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(f'Epoch no: {epoch}/{num_epochs-1}, Batch no: {i}/{total_batches}, Loss: {loss}')
    i+=1
  

Epoch no: 0/1, Batch no: 0/300, Loss: 0.8002278804779053
Epoch no: 0/1, Batch no: 1/300, Loss: 1.0299780368804932
Epoch no: 0/1, Batch no: 2/300, Loss: 1.0450252294540405
Epoch no: 0/1, Batch no: 3/300, Loss: 0.9137486219406128
Epoch no: 0/1, Batch no: 4/300, Loss: 1.0510627031326294
Epoch no: 0/1, Batch no: 5/300, Loss: 1.0226765871047974
Epoch no: 0/1, Batch no: 6/300, Loss: 0.9292600154876709
Epoch no: 0/1, Batch no: 7/300, Loss: 1.0380001068115234
Epoch no: 0/1, Batch no: 8/300, Loss: 0.946729838848114
Epoch no: 0/1, Batch no: 9/300, Loss: 1.0359259843826294
Epoch no: 0/1, Batch no: 10/300, Loss: 1.0117186307907104
Epoch no: 0/1, Batch no: 11/300, Loss: 0.8990718126296997
Epoch no: 0/1, Batch no: 12/300, Loss: 0.9889218807220459
Epoch no: 0/1, Batch no: 13/300, Loss: 1.007154107093811
Epoch no: 0/1, Batch no: 14/300, Loss: 0.9647420644760132
Epoch no: 0/1, Batch no: 15/300, Loss: 1.0852463245391846
Epoch no: 0/1, Batch no: 16/300, Loss: 1.1243287324905396
Epoch no: 0/1, Batch no: 1

In [ ]:
import torch
torch.save(model,'/content/gdrive/My Drive/outPut_1/model2.pth')

In [ ]:
from argparse import ArgumentParser, Namespace


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import pytorch_lightning as pl

class DataModule(pl.LightningDataModule):

    def __init__(self):
        super().__init__()
    
    def setup(self, stage):
        # called on each gpu
        n = 128
        x = torch.randn(n, 1, 64,64)
        data = list(zip(x,x))
        self.test  = DataLoader(data, batch_size=32)
        self.train = DataLoader(data, batch_size=32)
        self.val   = DataLoader(data, batch_size=32)
        
    def train_dataloader(self):
        return self.train

    def val_dataloader(self):
        return self.val

    def test_dataloader(self):
        return self.test
class SegModel(pl.LightningModule):
    def __init__(self):
        super(SegModel, self).__init__()
        self.batch_size = 4
        self.learning_rate = 1e-3
#         self.net = torchvision.models.segmentation.fcn_resnet50(pretrained = False, progress = True, num_classes = 19)
#         self.net = UNet(num_classes = 19, bilinear = False)
#         self.net = torchvision.models.segmentation.deeplabv3_resnet50(pretrained = False, progress = True, num_classes = 19)
        self.net = ENet(num_classes = 19)
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean = [0.35675976, 0.37380189, 0.3764753], std = [0.32064945, 0.32098866, 0.32325324])
        ])
        self.trainset = semantic_dataset(split = 'train', transform = self.transform)
        self.testset = semantic_dataset(split = 'test', transform = self.transform)
        
    def forward(self, x):
        return self.net(x)
    
    def training_step(self, batch, batch_nb) :
        img, mask = batch
        img = img.float()
        mask = mask.long()
        out = self.forward(img)
        loss_val = F.cross_entropy(out, mask, ignore_index = 250)
#         print(loss.shape)
        return {'loss' : loss_val}
    
    def configure_optimizers(self):
        opt = torch.optim.Adam(self.net.parameters(), lr = self.learning_rate)
        sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max = 10)
        return [opt], [sch]
    
    def train_dataloader(self):
        return DataLoader(self.trainset, batch_size = self.batch_size, shuffle = True)
    
    def test_dataloader(self):
        return DataLoader(self.testset, batch_size = 1, shuffle = True)

dm = DataModule()
model = SegModel()
trainer = pl.Trainer(gpus=1, 
                     distributed_backend="dp",
                     max_epochs=1,
                    )
trainer.fit(model, dm)

NameError: ignored

In [ ]:
!pip install pytorch_lightning